In [1]:
import glob
import re
import numpy as np
import pandas as pd
import clima_anom as ca
from netCDF4 import Dataset

import matplotlib.pyplot as plt

import matplotlib.cbook
import matplotlib.gridspec as gridspec

import cartopy
import cartopy.crs as ccrs

In [2]:
year_dir = '2020'

In [3]:
def closest_node(point, serie):
    dist_2 = (serie - point)**2
    return np.argmin(dist_2)

In [4]:
pluviometer_dir = '../../pluviometer/pluv_vale_lat_lon.csv'
list_pluviometer = pd.read_csv(pluviometer_dir,sep=';')
len_pluviometer = len(list_pluviometer)
list_pluviometer

,Name,Latitude,Longitude
0,RH-92-PLUVIOMETRO-EFVM,-19.649351,-43.159383
1,RH-84-PLUVIOMETRO-EFVM,-19.956498,-43.286721
2,RH-78-PLUVIOMETRO-EFVM,-19.746002,-43.022259
3,RH-72-PLUVIOMETRO-EFVM,-19.648011,-42.860899
4,RH-60-PLUVIOMETRO-EFVM,-19.313211,-42.362461
5,RH-50-PLUVIOMETRO-EFVM,-18.862691,-41.962364
6,RH-37-PLUVIOMETRO-EFVM,-19.203117,-41.399608
7,RH-23-PLUVIOMETRO-EFVM,-19.542816,-40.689570
8,RH-210V-PLUVIOMETRO-EFVM,-19.873132,-43.778078
9,RH-203-PLUVIOMETRO-EFVM,-19.927519,-43.403119


In [5]:
IMERG_files = glob.glob('/run/media/arturo/Arturo/Data/IMERG/raw_30min/'+year_dir+'/*.nc4')
IMERG_files = sorted(IMERG_files)
len_IMERG = len(IMERG_files)
print(f'Year {year_dir} has {len_IMERG} files in directory')
IMERG_files[0:5]

Year 2020 has 17568 files in directory


['/run/media/arturo/Arturo/Data/IMERG/raw_30min/2020/3B-HHR.MS.MRG.3IMERG.20200101-S000000-E002959.0000.V06B.HDF5.nc4',
 '/run/media/arturo/Arturo/Data/IMERG/raw_30min/2020/3B-HHR.MS.MRG.3IMERG.20200101-S003000-E005959.0030.V06B.HDF5.nc4',
 '/run/media/arturo/Arturo/Data/IMERG/raw_30min/2020/3B-HHR.MS.MRG.3IMERG.20200101-S010000-E012959.0060.V06B.HDF5.nc4',
 '/run/media/arturo/Arturo/Data/IMERG/raw_30min/2020/3B-HHR.MS.MRG.3IMERG.20200101-S013000-E015959.0090.V06B.HDF5.nc4',
 '/run/media/arturo/Arturo/Data/IMERG/raw_30min/2020/3B-HHR.MS.MRG.3IMERG.20200101-S020000-E022959.0120.V06B.HDF5.nc4']

In [6]:
lat_new = []
lon_new = []

for plu in range(len_pluviometer):

    data = ca.read_netcdf(IMERG_files[0],0)
    lat = data['lat'].data
    lon = data['lon'].data

    lat_index = closest_node(np.round(list_pluviometer['Latitude'][plu],2), lat)
    lon_index = closest_node(np.round(list_pluviometer['Longitude'][plu],2), lon)

    lat_new.append(lat[lat_index])
    lon_new.append(lon[lon_index])

pos_IMERG = pd.DataFrame({'Lat_PLUV':list_pluviometer['Latitude'],'Lon_PLUV':list_pluviometer['Longitude'],'Lat_IMERG':lat_new,'Lon_IMERG':lon_new})
pos_IMERG.head(15)

,Lat_PLUV,Lon_PLUV,Lat_IMERG,Lon_IMERG
0,-19.649351,-43.159383,-19.650000,-43.150002
1,-19.956498,-43.286721,-19.950001,-43.250000
2,-19.746002,-43.022259,-19.750000,-43.049999
3,-19.648011,-42.860899,-19.650000,-42.849998
4,-19.313211,-42.362461,-19.350000,-42.349998
5,-18.862691,-41.962364,-18.850000,-41.950001
6,-19.203117,-41.399608,-19.250000,-41.450001
7,-19.542816,-40.689570,-19.549999,-40.650002
8,-19.873132,-43.778078,-19.850000,-43.750000
9,-19.927519,-43.403119,-19.950001,-43.450001


In [7]:
#export dataframe to csv
name_out = '../../export/Lat_Lon_PLUV_IMERG.csv'
print(f'Exporting data to {name_out}')
# pos_IMERG.to_csv(name_out,sep=',',index=False)

Exporting data to ../../export/Lat_Lon_PLUV_IMERG.csv


In [8]:
pre_export = None

year = []
month = []
day = []
hour = []
minute = []

for nn in range(len_IMERG):

    if nn % 1000 == 0:
        print(f'{nn} of {len_IMERG}')

    data_ = ca.read_netcdf(IMERG_files[nn],0)
    tim = data_['time'].data
    pre = data_['precipitationCal'][0,:,:].T.data

    yy = tim[0].year
    mm = tim[0].month
    dd = tim[0].day
    hh = tim[0].hour
    mi = tim[0].minute

    year.append(yy)
    month.append(mm)
    day.append(dd)
    hour.append(hh)
    minute.append(mi)

    if pre_export is None:
        
        lat = data_['lat'].data
        lon = data_['lon'].data
        
        pre_export = np.zeros([len_IMERG,len_pluviometer])

    for plu in range(len_pluviometer):

        lat_IMERG = pos_IMERG['Lat_IMERG'][plu]
        lon_IMERG = pos_IMERG['Lon_IMERG'][plu]

        lat_index = np.where(lat == lat_IMERG)[0][0]
        lon_index = np.where(lon == lon_IMERG)[0][0]

        pre_export[nn,plu] = pre[lat_index,lon_index]

        # for plu in range(len_pluviometer):

        #     print(f'Pluviometer: {plu}')

        #     lat_index = closest_node(np.round(list_pluviometer['Latitude'][plu],2), lat)
        #     lon_index = closest_node(np.round(list_pluviometer['Longitude'][plu],2), lon)

        # pre_export[nn,plu] = pre[lat_index,lon_index]

0 of 17568
1000 of 17568
2000 of 17568
3000 of 17568
4000 of 17568
5000 of 17568
6000 of 17568
7000 of 17568
8000 of 17568
9000 of 17568
10000 of 17568
11000 of 17568
12000 of 17568
13000 of 17568
14000 of 17568
15000 of 17568
16000 of 17568
17000 of 17568


In [9]:
names_pluv = []
for t in range(len_pluviometer):
    tmp = list_pluviometer['Name'][t].replace('-PLUVIOMETRO-EFVM','')
    names_pluv.append(tmp)

In [10]:
salida = pd.DataFrame(data=pre_export,columns=names_pluv)
salida.insert(loc=0, column='year', value=year)
salida.insert(loc=1, column='month', value=month)
salida.insert(loc=2, column='day', value=day)
salida.insert(loc=3, column='hour', value=hour)
salida.insert(loc=4, column='minute', value=minute)

salida.head(5)

,year,month,day,hour,minute,RH-92,RH-84,RH-78,RH-72,RH-60,RH-50,RH-37,RH-23,RH-210V,RH-203,RH-18V,RH-14,RH-13I,RH-07,PT-TU-D6
0,2020,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020,1,1,0,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020,1,1,1,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,1,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
name_out = '../../export/IMERG_'+year_dir+'_01_'+year_dir+'_12_30min.csv'
print(f'Exporting data to {name_out}')
salida.to_csv(name_out,sep=',',index=False)

Exporting data to ../../export/IMERG_2020_01_2020_12_30min_new.csv
